In [8]:
import matplotlib.pyplot as plt
import numpy as np
import triangle
import plotly.graph_objects as go
from shapely.geometry import LineString, MultiLineString, Point, MultiPoint, GeometryCollection
from ctod.core.terrain_request import TerrainRequest
from ctod.handlers.terrain import TerrainHandler

from ctod.core.cog import download_tile
from ctod.core.utils import get_tms, get_tile_bounds
from ctod.core.mesh import generate_mesh

%pip install --upgrade nbformat

tms = get_tms()
geotiff_path = "/home/time/Downloads/cog/dtm_cog.tif"

Note: you may need to restart the kernel to use updated packages.


In [149]:
# north tile
north_x, north_y, north_z = 67501, 10799, 16
north = download_tile(tms, north_x, north_y, north_z, geotiff_path)
tile_bounds_north = get_tile_bounds(tms, north_x, north_y, north_z)

# south tile
south_x, south_y, south_z = 67501, 10798, 16
south = download_tile(tms, south_x, south_y, south_z, geotiff_path)
tile_bound_south = get_tile_bounds(tms, south_x, south_y, south_z)

mesh_error = 1.5
north_vertices_v2, north_triangles_v2 = generate_mesh(north.data[0], mesh_error)
south_vertices_v2, south_triangles_v2 = generate_mesh(south.data[0], mesh_error)

In [10]:
def get_edge(vertices, direction):
    if direction == "north":
        return vertices[vertices[:, 1] == 255]
    elif direction == "south":
        return vertices[vertices[:, 1] == 0]
    elif direction == "east":
        return vertices[vertices[:, 0] == 255]
    elif direction == "west":
        return vertices[vertices[:, 0] == 0]
    else:
        raise ValueError("Invalid direction:", direction)

In [150]:
def calculate_normals(vertices, triangles):
    vertex_normals = np.zeros_like(vertices)

    # Iterate over each triangle
    for i in range(triangles.shape[0]):
        # Get the indices of the vertices for the current triangle
        vertex_indices = triangles[i]

        # Calculate the normal of the current face
        v0 = vertices[vertex_indices[0]]
        v1 = vertices[vertex_indices[1]]
        v2 = vertices[vertex_indices[2]]
        face_normal = np.cross(v1 - v0, v2 - v0)


        # Accumulate the face normal to the vertex normals
        vertex_normals[vertex_indices] += face_normal

    # Calculate the lengths of the vertex normals
    norm_lengths = np.linalg.norm(vertex_normals, axis=1)

    # Check if the length is non-zero before dividing
    non_zero_lengths = norm_lengths > 0
    vertex_normals[non_zero_lengths] /= norm_lengths[non_zero_lengths][:, np.newaxis]
    
    return vertex_normals


In [169]:

# Get the north tile south edge vertices
north_edge_vertices = get_edge(north_vertices_v2, "south")

# set the y of the edge vertices to 255
north_edge_vertices[:, 1] = 255

# Add the edges of the north tile to the south tile
south_all_vertices = np.concatenate((south_vertices_v2, north_edge_vertices), axis=0)


# Make vertices 2D
south_all_vertices_2d = south_all_vertices[:, :2]
south_all_vertices_2d = np.unique(south_all_vertices_2d, axis=0)


# Triangulate with constraint on tile border
triangulation_south = triangle.triangulate(
    {'vertices': south_all_vertices_2d, 'segments': [[0, 0], [255, 0], [255, 255], [0, 255], [0, 0]]}
)

south_vertices_new = triangulation_south["vertices"]
south_triangles_new = triangulation_south["triangles"]

# Resample the height for vertices from the cog
# Sample z-values from image_data based on x and y coordinates
x_indices = south_vertices_new[:, 0].astype(int)
y_indices = south_vertices_new[:, 1].astype(int)

# Check if y-coordinate is on the edge
on_edge = (y_indices == 255)

# Calculate z-values
z_values_south = south.data[0][y_indices, x_indices]

# For vertices on the edge, interpolate between south.data[0] and north_data[0]
z_values_north = north.data[0][0, x_indices]

z_values_on_edge = (z_values_north + z_values_south) / 2.0

# Combine z-values for non-edge and edge vertices
z_values = np.where(on_edge, z_values_on_edge, z_values_south)

# Add the sampled z-values as the third column in modified_array
south_vertices_new = np.column_stack((south_vertices_new, z_values))

south_normals = calculate_normals(south_vertices_new, south_triangles_new)

In [170]:

# Get the south tile north edge vertices
south_edge_vertices = get_edge(south_vertices_v2, "north")

south_edge_vertices[:, 1] = 0

# Add the edges of the south tile to the north tile
north_all_vertices = np.concatenate((north_vertices_v2, south_edge_vertices), axis=0)



# Make vertices 2D
north_all_vertices_2d = north_all_vertices[:, :2]
north_all_vertices_2d = np.unique(north_all_vertices_2d, axis=0)

# Triangulate with constraint on tile border
triangulation_north = triangle.triangulate(
    {'vertices': north_all_vertices_2d, 'segments': [[0, 0], [255, 0], [255, 255], [0, 255], [0, 0]]}
)

north_vertices_new = triangulation_north["vertices"]
north_triangles_new = triangulation_north["triangles"]

# Resample the height for vertices from the cog
# Sample z-values from image_data based on x and y coordinates
x_indices = north_vertices_new[:, 0].astype(int)
y_indices = north_vertices_new[:, 1].astype(int)

# Check if y-coordinate is on the edge
on_edge = (y_indices == 0)

# Calculate z-values
z_values_north = north.data[0][y_indices, x_indices]

# For vertices on the edge, interpolate between south.data[0] and north_data[0]
z_values_south = south.data[0][255, x_indices]
z_values_on_edge = (z_values_north + z_values_south) / 2.0

# Combine z-values for non-edge and edge vertices
z_values = np.where(on_edge, z_values_on_edge, z_values_north)

# Add the sampled z-values as the third column in modified_array
north_vertices_new = np.column_stack((north_vertices_new, z_values))

north_normals = calculate_normals(north_vertices_new, north_triangles_new)



In [144]:
def angle_between_vectors(v1, v2):
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)

    if norm_v1 == 0 or norm_v2 == 0:
        # Handle the case where either vector is a zero vector
        return 0.0

    cos_theta = np.dot(v1, v2) / (norm_v1 * norm_v2)
    return np.arccos(np.clip(cos_theta, -1.0, 1.0))

In [138]:
# Offset the y-coordinates of south_vertices by max_y_north
vertices_north_tile_offset = north_vertices_new.copy()
vertices_north_tile_offset[:, 1] += 255

# Combine vertices
all_vertices_new = np.concatenate((south_vertices_new, vertices_north_tile_offset), axis=0)

# Offset the triangle indices for south_triangles and combine
triangles_north_tile_offset = north_triangles_new + len(south_vertices_new)
all_triangles_new = np.concatenate((south_triangles_new, triangles_north_tile_offset), axis=0)

x, y, z = all_vertices_new[:, 0], all_vertices_new[:, 1], all_vertices_new[:, 2]
i, j, k = all_triangles_new[:, 0], all_triangles_new[:, 1], all_triangles_new[:, 2]
color_scale = z
layout = go.Layout(scene=dict(aspectmode="data"))

light_source = np.array([1, 1, 1], dtype=np.float64)
light_source /= np.linalg.norm(light_source)

south_normals /= np.linalg.norm(south_normals, axis=1, keepdims=True).astype(np.float64)
north_normals /= np.linalg.norm(north_normals, axis=1, keepdims=True).astype(np.float64)
boundary_vertices_index = np.where(vertices_north_tile_offset[:, 1] == max(south_vertices_new[:, 1]))[0]
south_normals[boundary_vertices_index] = north_normals[boundary_vertices_index]

# Combine the adjusted normals
normals = np.concatenate((south_normals, north_normals), axis=0)

# Calculate the shading based on the cosine of the angle between normals and light source
shading = np.clip(np.dot(normals, light_source), 0, 1)

# Create the mesh with shading based on normals
mesh = go.Mesh3d(
    x=x, y=y, z=z, i=i, j=j, k=k,
    intensity=shading,
    colorscale='Viridis', opacity=1.0, flatshading=False  # Set flatshading to False for smooth shading
)

# Add wireframe to the figure
wireframe = go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, color='rgba(0,0,0,0.2)')

# Add mesh and wireframe to the figure
fig = go.Figure(data=[mesh, wireframe], layout=layout)

# Visualize normals as lines
normal_length = 10  # Adjust the length of the normal lines as needed

for idx, vertex in enumerate(all_vertices_new):
    normal_end = vertex + normal_length * normals[idx]
    fig.add_trace(go.Scatter3d(
        x=[vertex[0], normal_end[0]],
        y=[vertex[1], normal_end[1]],
        z=[vertex[2], normal_end[2]],
        mode='lines',
        line=dict(color='red', width=2),
    ))
    
# Adjust the size of the figure
fig.update_layout(width=1000, height=800)  # Adjust the width and height as needed

# Display the figure
fig.show()

In [54]:
def fix_normals_at_boundary(vertices, normals, threshold=1e-6):
    """
    Fix normals at the boundary by adjusting normals for vertices with the same location.

    Parameters:
    - vertices (numpy.ndarray): Array of vertex coordinates (n x 3).
    - normals (numpy.ndarray): Array of normals corresponding to each vertex (n x 3).
    - threshold (float): Threshold for considering vertices as identical.

    Returns:
    - numpy.ndarray: Updated normals.
    """

    # Find indices of vertices with the same location
    unique_indices, counts = np.unique(vertices, axis=0, return_counts=True)
    duplicate_indices = unique_indices[counts > 1]

    # Adjust normals for vertices with the same location
    for duplicate_vertex in duplicate_indices:
        indices = np.where(np.all(vertices == duplicate_vertex, axis=1))[0]
        if len(indices) > 1:
            print("test")
            # Calculate the average normal for the duplicate vertices
            avg_normal = np.mean(normals[indices], axis=0)
            avg_normal /= np.linalg.norm(avg_normal)  # Normalize the average normal

            # Update normals for the duplicate vertices
            normals[indices] = avg_normal

    return normals

In [145]:
def interpolate_normals(all_vertices, all_normals, positions, normalized_vertex_normals):
    if all_vertices is not None and all_normals is not None:
        all_vertices = all_vertices.reshape(-1, 3)
        #all_vertices = to_ecef(all_vertices, ellipsoid=self.ellipsoid)

        for i in range(len(positions)):
            vertex = positions[i]
            #print(vertex)

            xy_all_vertices = all_vertices[:, :2]
            xy_vertex = vertex[:2]
            #match_indices = np.where(np.all(all_vertices == vertex, axis=1))[0]
            match_indices = np.where(np.all(xy_all_vertices == xy_vertex, axis=1))[0]

            if len(match_indices) > 0:
                region_vertex_index = match_indices[0]

                # Check for zero vectors before calculating the angle
                if np.any(normalized_vertex_normals[i] == 0) or np.any(all_normals[region_vertex_index] == 0):
                    # Handle special case: skip interpolation or set a default value
                    #print("Skipping interpolation due to zero vectors.")
                    continue

                angle = angle_between_vectors(
                    normalized_vertex_normals[i], all_normals[region_vertex_index]
                )
                
                #print(angle)

                # Check for NaN result before proceeding with interpolation
                if np.isnan(angle):
                    # Handle special case: skip interpolation or set a default value
                    print("Skipping interpolation due to NaN angle.")
                    continue

                weight = np.sin(angle)
                
                #print(f"Before interpolation - Vertex {i}: {normalized_vertex_normals[i]}")

                normalized_vertex_normals[i] = (normalized_vertex_normals[i] + all_normals[region_vertex_index]) / 2
                #normalized_vertex_normals[i] = (
                #    normalized_vertex_normals[i] + all_normals[region_vertex_index] * weight
                #) / (1.0 + weight)
                
                # test set normalized height
                #positions[i][2] = (positions[i][2] + all_normals[region_vertex_index][2]) / 2

    return normalized_vertex_normals


In [103]:
def calculate_normalized_normals(vertices, triangles, other_vertices, other_triangles):
    vertex_normals = calculate_normals(vertices, triangles)
    vertex_normals = vertex_normals.reshape(-1, 3)
    all_vertices = []
    all_normals = []

    #all_vertices.append(vertices)
    #all_normals.append(vertex_normals)
            
    def add_region(region_vertices, region_triangles):
        if region_vertices is not None and region_triangles is not None:
            region_vertices = region_vertices.reshape(-1, 3)
            region_normals = calculate_normals(region_vertices, region_triangles)
            all_vertices.append(region_vertices)
            all_normals.append(region_normals)
            
    add_region(other_vertices, other_triangles)
    all_vertices = np.vstack(all_vertices) if all_vertices else None
    all_normals = np.vstack(all_normals) if all_normals else None
    
    vertex_normals = interpolate_normals(all_vertices, all_normals, vertices, vertex_normals)
    
    return vertex_normals

In [171]:

# working with image coords here instead of ecef, we need to move the north tile up by 256
temp_north_vertices = north_vertices_new.copy()
temp_north_vertices[:, 1] += 255
north_normals_normalized = calculate_normalized_normals(temp_north_vertices, north_triangles_new, south_vertices_new, south_triangles_new)
south_normals_normalized = calculate_normalized_normals(south_vertices_new, south_triangles_new, temp_north_vertices, north_triangles_new)
#print(north_normals_normalized)


In [172]:
# Offset the y-coordinates of south_vertices by max_y_north
vertices_north_tile_offset = north_vertices_new.copy()
vertices_north_tile_offset[:, 1] += 255

# Combine vertices
all_vertices_new = np.concatenate((south_vertices_new, vertices_north_tile_offset), axis=0)

# Offset the triangle indices for south_triangles and combine
triangles_north_tile_offset = north_triangles_new + len(south_vertices_new)
all_triangles_new = np.concatenate((south_triangles_new, triangles_north_tile_offset), axis=0)

x, y, z = all_vertices_new[:, 0], all_vertices_new[:, 1], all_vertices_new[:, 2]
i, j, k = all_triangles_new[:, 0], all_triangles_new[:, 1], all_triangles_new[:, 2]
color_scale = z
layout = go.Layout(scene=dict(aspectmode="data"))

light_source = np.array([1, 1, 1], dtype=np.float64)
light_source /= np.linalg.norm(light_source)

#south_normals /= np.linalg.norm(south_normals, axis=1, keepdims=True).astype(np.float64)
#north_normals /= np.linalg.norm(north_normals, axis=1, keepdims=True).astype(np.float64)
#boundary_vertices_index = np.where(vertices_north_tile_offset[:, 1] == max(south_vertices_new[:, 1]))[0]
#south_normals[boundary_vertices_index] = north_normals[boundary_vertices_index]

# Combine the adjusted normals
normals = np.concatenate((south_normals_normalized, north_normals_normalized), axis=0)
#all_normals_fixed = fix_normals_at_boundary(all_vertices_new, normals)


# Calculate the shading based on the cosine of the angle between normals and light source
shading = np.clip(np.dot(normals, light_source), 0, 1)

# Create the mesh with shading based on normals
mesh = go.Mesh3d(
    x=x, y=y, z=z, i=i, j=j, k=k,
    intensity=shading,
    colorscale='Viridis', opacity=1.0, flatshading=False  # Set flatshading to False for smooth shading
)

# Add wireframe to the figure
wireframe = go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, color='rgba(0,0,0,0.2)')

# Add mesh and wireframe to the figure
fig = go.Figure(data=[mesh, wireframe], layout=layout)

# Visualize normals as lines
normal_length = 10  # Adjust the length of the normal lines as needed

for idx, vertex in enumerate(all_vertices_new):
    normal_end = vertex + normal_length * normals[idx]
    fig.add_trace(go.Scatter3d(
        x=[vertex[0], normal_end[0]],
        y=[vertex[1], normal_end[1]],
        z=[vertex[2], normal_end[2]],
        mode='lines',
        line=dict(color='red', width=2),
    ))
    
# Adjust the size of the figure
fig.update_layout(width=1000, height=800)  # Adjust the width and height as needed

# Display the figure
fig.show()